In [1]:
import numpy as np
import pandas as pd
import re
from simulation import *
import sklearn
import sklearn.model_selection
import sklearn.linear_model
from data import *
from models import *

In [5]:
print('')
print(regSeasDetailedTotals.head())


  Season TeamID     G  Wins  Losses     Pts      PA    FGM     FGA   FGM3  \
0   2003   1104  28.0  17.0    11.0  1940.0  1820.0  673.0  1601.0  178.0   
1   2003   1272  29.0  23.0     6.0  2161.0  1909.0  762.0  1740.0  203.0   
2   2003   1266  28.0  23.0     5.0  2195.0  1895.0  762.0  1575.0  162.0   
3   2003   1296  31.0  17.0    14.0  2158.0  2164.0  755.0  1645.0  195.0   
4   2003   1400  28.0  22.0     6.0  2208.0  1923.0  784.0  1748.0  164.0   

   ...      TR      RebPG      AstPG       TOPG     StlPG     BlkPG  \
0  ...  1050.0  37.500000  12.107143  13.285714  6.607143  3.785714   
1  ...  1161.0  40.034483  16.620690  13.793103  7.379310  5.068966   
2  ...  1041.0  37.178571  16.321429  13.571429  6.000000  3.642857   
3  ...  1105.0  35.645161  12.677419  17.000000  7.612903  3.612903   
4  ...  1185.0  42.321429  14.500000  13.428571  6.392857  3.857143   

        PFPG  PtsDif   PtsPGDif       ATR  
0  18.035714   120.0   4.285714  0.911290  
1  18.758621   252.0 

In [3]:
seedResults = createSeedResultsDF(seeds, tourneyCompactResults)
masterCompact = createMasterDF(seedResults, regSeasCompactTotals)
masterDetailed = createMasterDF(seedResults, regSeasDetailedTotals)

In [5]:
#trying models

highSeedWinsAcc = tourneySimVsActual(list(range(1985, 2020)), highSeedWins, masterCompact)
betterRecordWinsAcc = tourneySimVsActual(list(range(1985, 2020)), highSeedWins, masterCompact)
betterPtsDifWinsAcc = tourneySimVsActual(list(range(1985, 2020)), betterPtsDifWins, masterCompact)
betterPtsDifPGWinsAcc = tourneySimVsActual(list(range(1985, 2020)), betterPtsDifPGWins, masterCompact)
betterFTPercWinsAcc = tourneySimVsActual(list(range(2003, 2020)), betterFTPercWins, masterDetailed)
betterAstTORatioWinsAcc = tourneySimVsActual(list(range(2003, 2020)), betterAstTORatioWins, masterDetailed)

print('High seed wins model accuracy: ', highSeedWinsAcc)
print('Better record wins model accuracy: ', betterRecordWinsAcc)
print('Better point differential wins model accuracy: ', betterPtsDifWinsAcc)
print('Better point differential per game wins model accuracy: ', betterPtsDifPGWinsAcc)
print('Better free throw percentage wins model accuracy: ', betterFTPercWinsAcc)
print('Better assist/turnover ratio wins model accuracy: ', betterAstTORatioWinsAcc)

High seed wins model accuracy:  0.6410484229231452
Better record wins model accuracy:  0.6406041759218125
Better point differential wins model accuracy:  0.5832963127498889
Better point differential per game wins model accuracy:  0.5726343847179032
Better free throw percentage wins model accuracy:  0.3506726457399103
Better assist/turnover ratio wins model accuracy:  0.473542600896861


In [4]:
### logistic regression preliminary analysis

detailedLogRegDF = createDetailedLogRegDF(masterDetailed)

y = detailedLogRegDF['gameOutcome'].to_numpy()

logRegCols1 = ['SelfNumSeed', 'OppNumSeed']
logRegDF2 = detailedLogRegDF[:]
logRegDF2['NumSeedDiff'] = logRegDF2['SelfNumSeed'] - logRegDF2['OppNumSeed']
x1 = detailedLogRegDF[logRegCols1].to_numpy()
x2 = np.expand_dims(logRegDF2['NumSeedDiff'].to_numpy(), axis = 1)

x1_train, x1_test, y1_train, y1_test = sklearn.model_selection.train_test_split(x1, y, test_size = 0.25, random_state = 1)
x2_train, x2_test, y2_train, y2_test = sklearn.model_selection.train_test_split(x2, y, test_size = 0.25, random_state = 1)
print(x1_train.shape, x1_test.shape, y1_train.shape, y1_test.shape)
print(x2_train.shape, x2_test.shape, y2_train.shape, y2_test.shape)

logreg1 = sklearn.linear_model.LogisticRegression()
logreg2 = sklearn.linear_model.LogisticRegression()

jack = logreg1.fit(x1_train, y1_train)
karisch = logreg2.fit(x2_train, y2_train)

jackpred = logreg1.predict(x1_test)
karischpred = logreg2.predict(x2_test)

(jackpred == karischpred).mean()

(836, 2) (279, 2) (836,) (279,)
(836, 1) (279, 1) (836,) (279,)


0.992831541218638

In [64]:
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def AIC(y, ySigmoid, k):
    residuals = y - ySigmoid
    RSS = (residuals ** 2).sum()
    AIC = 2*k - 2*np.log(RSS)
    return AIC

def AICcomparison(AIC1, AIC2):
    if AIC1 <= AIC2:
        return np.exp((AIC1 - AIC2) / 2)
    else:
        return np.exp((AIC2 - AIC1) / 2)

In [51]:
out = []

for i in range(len(x2_test)):
    iRaw = float(x2_test[i]) * float(karisch.coef_)
    iSig = sigmoid(iRaw)
    iPred = int(round(iSig))
    out.append(iSig)

In [66]:
masterDetailed.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WSection', 'WNumSeed', 'LSection', 'LNumSeed', 'WG', 'WWins',
       'WLosses', 'WPts', 'WPA', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM',
       'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'WRecord',
       'WPtsPG', 'WFGPerc', 'WFG3Perc', 'WFTPerc', 'WTR', 'WRebPG', 'WAstPG',
       'WTOPG', 'WStlPG', 'WBlkPG', 'WPFPG', 'WPtsDif', 'WPtsPGDif', 'WATR',
       'LG', 'LWins', 'LLosses', 'LPts', 'LPA', 'LFGM', 'LFGA', 'LFGM3',
       'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk',
       'LPF', 'LRecord', 'LPtsPG', 'LFGPerc', 'LFG3Perc', 'LFTPerc', 'LTR',
       'LRebPG', 'LAstPG', 'LTOPG', 'LStlPG', 'LBlkPG', 'LPFPG', 'LPtsDif',
       'LPtsPGDif', 'LATR'],
      dtype='object')